In [ ]:
import sys

!{sys.executable} -m pip install pygame

import pygame
import math
import heapq



#colores representacion
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
GREY = (200, 200, 200)


ROWS, COLS = 20, 20
WIDTH, HEIGHT = 600, 600
CELL_SIZE = WIDTH // COLS

pygame.init()
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Algoritmo A* Interactivo")


class Node:
    def inicializar(self, row, col, width, total_rows):
        self.row = row
        self.col = col
        self.x= row*width
        self.y= col*width
        self.color = WHITE
        self.neighbors= []
        self.width= width
        self.total_rows= total_rows
        self.risk= 0

    
    def is_start(self):
        return self.color == GREEN
    
    def is_end(self):
        return self.color == BLUE
    
    def is_barrier(self):
        return self.color == BLACK
    
    def is_risky(self):
        return self.color == YELLOW
    
    def reset(self):
        self.color = WHITE
        self.risk = 0
    
    def make_start(self):
        self.color = GREEN
    
    def make_end(self):
        self.color = BLUE
    
    def make_barrier(self):
        self.color = BLACK
    
    def make_risky(self):
        self.color = YELLOW
        self.risk = 5  #penalizacion

    def draw(self, win):
        pygame.draw.rect(win, self.color, (self.x, self.y, CELL_SIZE, CELL_SIZE))
        if self.is_barrier():
            pygame.draw.line(win, RED, (self.x, self.y), (self.x + CELL_SIZE, self.y + CELL_SIZE), 3)
            pygame.draw.line(win, RED, (self.x + CELL_SIZE, self.y), (self.x, self.y + CELL_SIZE), 3)
        elif self.is_risky():
            pygame.draw.polygon(win, BLACK, [(self.x + CELL_SIZE//2, self.y + 5), (self.x + 5, self.y + CELL_SIZE - 5), (self.x + CELL_SIZE - 5, self.y + CELL_SIZE - 5)], 2)
    
    def update_neighbors(self, grid):
        self.neighbors = []
        directions = [
            (0, 1), (0, -1), (1, 0), (-1, 0),  #Ortogonales
            (1, 1), (-1, -1), (1, -1), (-1, 1) #Diagonales
        ]
        for dr, dc in directions:
            r, c = self.row + dr, self.col + dc
            if 0 <= r < ROWS and 0 <= c < COLS and not grid[r][c].is_barrier():
                self.neighbors.append(grid[r][c])


    def h(p1, p2):
        x1, y1 = p1
        x2, y2 = p2
        return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
    
    def reconstruct_path(current, draw):
        while current.parent:
            current = current.parent
            current.make_path()
            draw()

    def a_star(draw, grid, start, end):
        count = 0
        open_set = []
        heapq.heappush(open_set, (0, count, start))
        came_from = {}
        
        g_score = {node: float("inf") for row in grid for node in row}
        g_score[start] = 0
        f_score = {node: float("inf") for row in grid for node in row}
        f_score[start] = h((start.row, start.col), (end.row, end.col))

        open_set_hash = {start}
        
        while open_set:
            current = heapq.heappop(open_set)[2]
            open_set_hash.remove(current)

            if current == end:
                reconstruct_path(current, draw)
                end.make_end()
                return True
            
            for neighbor in current.neighbors:
                temp_g_score = g_score[current] + 1 + neighbor.risk  #Penalizacion
                
                if temp_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = temp_g_score
                    f_score[neighbor] = temp_g_score + h((neighbor.row, neighbor.col), (end.row, end.col))
                    neighbor.parent = current

                    if neighbor not in open_set_hash:
                        count += 1
                        heapq.heappush(open_set, (f_score[neighbor], count, neighbor))
                        open_set_hash.add(neighbor)
            
            draw()
        return False


    




[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip
